# 02 - Tendencia Regulatoria (REN 414 x REN 1000)

Analise pre-2022 vs pos-2022 com base em indicadores anuais consolidados.

In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd()
if not (ROOT / 'data').exists():
    ROOT = ROOT.parent
ANALYSIS_DIR = ROOT / 'data' / 'processed' / 'analysis'

fato = pd.read_parquet(ANALYSIS_DIR / 'fato_indicadores_anuais.parquet')
kpi = pd.read_parquet(ANALYSIS_DIR / 'kpi_regulatorio_anual.parquet')

In [ ]:
kpi = kpi[kpi['ano'] <= 2023].copy()
kpi

In [ ]:
# Pre vs pos
pre = kpi[kpi['periodo_regulatorio'] == 'pre_2022']
pos = kpi[kpi['periodo_regulatorio'] == 'pos_2022']

pre_taxa = pre['qtd_fora_prazo'].sum() / pre['qtd_serv'].sum()
pos_taxa = pos['qtd_fora_prazo'].sum() / pos['qtd_serv'].sum()
pre_comp = pre['compensacao_rs'].sum()
pos_comp = pos['compensacao_rs'].sum()

pd.DataFrame({
    'periodo': ['pre_2022', 'pos_2022'],
    'taxa_fora_prazo': [pre_taxa, pos_taxa],
    'compensacao_rs_total': [pre_comp, pos_comp],
})

In [ ]:
# Tendencia anual de taxa e compensacao
fig, ax = plt.subplots(1, 2, figsize=(14, 4))
ax[0].plot(kpi['ano'], kpi['taxa_fora_prazo'], marker='o')
ax[0].set_title('Taxa fora do prazo')
ax[0].set_xlabel('Ano')
ax[0].grid(True, alpha=0.3)

ax[1].plot(kpi['ano'], kpi['compensacao_rs'], marker='o')
ax[1].set_title('Compensacao total (R$)')
ax[1].set_xlabel('Ano')
ax[1].grid(True, alpha=0.3)

plt.tight_layout()

In [ ]:
# Recorte por classe/localidade
recorte = (
    fato[fato['ano'].between(2011, 2023)]
    .groupby(['ano', 'classe_local'], as_index=False)
    .agg(qtd_serv=('qtd_serv', 'sum'), qtd_fora_prazo=('qtd_fora_prazo', 'sum'))
)
recorte['taxa_fora_prazo'] = recorte['qtd_fora_prazo'] / recorte['qtd_serv']
recorte.sort_values(['ano', 'classe_local']).head(30)